# LSTM in Keras

* adapted from [adventuresinmachinelearning](http://adventuresinmachinelearning.com/keras-lstm-tutorial/)
* sequence to sequence text prediction

In [19]:
from __future__ import print_function
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout, TimeDistributed, Reshape, Lambda
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam, SGD
from keras import backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import argparse
import pdb

You need to get the data: `wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz` then unzip it. 

In [20]:
# change the path to point to the data, if needed
data_path = "simple-examples/data"

## load, shape data

In [21]:
def read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        return f.read().replace("\n", "<eos>").split()


def build_vocab(filename):
    data = read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))

    return word_to_id


def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]


def load_data():
    # get the data paths
    train_path = os.path.join(data_path, "ptb.train.txt")
    valid_path = os.path.join(data_path, "ptb.valid.txt")
    test_path = os.path.join(data_path, "ptb.test.txt")

    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_path)
    train_data = file_to_word_ids(train_path, word_to_id)
    valid_data = file_to_word_ids(valid_path, word_to_id)
    test_data = file_to_word_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    return train_data, valid_data, test_data, vocabulary, reversed_dictionary

train_data, valid_data, test_data, vocabulary, reversed_dictionary = load_data()


## helper class

In [22]:
class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set
        self.skip_step = skip_step

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                x[i, :] = self.data[self.current_idx:self.current_idx + self.num_steps]
                temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
                # convert all of temp_y into a one hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
            yield x, y

## model params

In [26]:
num_steps = 30
batch_size = 20
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)

hidden_size = 500
num_epochs = 20
use_dropout=True

## define model

In [27]:
model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))

optimizer = Adam()
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['categorical_accuracy'])

print(model.summary())
checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}.hdf5', verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 30, 500)           5000000   
_________________________________________________________________
lstm_5 (LSTM)                (None, 30, 500)           2002000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 30, 500)           2002000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 30, 500)           0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 10000)         5010000   
_________________________________________________________________
activation_3 (Activation)    (None, 30, 10000)         0         
Total params: 14,014,000
Trainable params: 14,014,000
Non-trainable params: 0
________________________________________________________________

## train model

In [28]:
model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=len(valid_data)//(batch_size*num_steps), callbacks=[checkpointer])
    # model.fit_generator(train_data_generator.generate(), 2000, num_epochs,
    #                     validation_data=valid_data_generator.generate(),
    #                     validation_steps=10)
model.save(data_path + "final_model.hdf5")

Epoch 1/20
1549/1549 [==============================] - 1854s 1s/step - loss: 6.7868 - categorical_accuracy: 0.0723 - val_loss: 6.0418 - val_categorical_accuracy: 0.1366

Epoch 00001: saving model to simple-examples/data/model-01.hdf5
Epoch 2/20
1549/1549 [==============================] - 1623s 1s/step - loss: 5.7861 - categorical_accuracy: 0.1624 - val_loss: 5.4721 - val_categorical_accuracy: 0.1866

Epoch 00002: saving model to simple-examples/data/model-02.hdf5
Epoch 3/20
1549/1549 [==============================] - 1648s 1s/step - loss: 5.3463 - categorical_accuracy: 0.1951 - val_loss: 5.2200 - val_categorical_accuracy: 0.2089

Epoch 00003: saving model to simple-examples/data/model-03.hdf5
Epoch 4/20
1549/1549 [==============================] - 1611s 1s/step - loss: 5.0771 - categorical_accuracy: 0.2149 - val_loss: 5.1058 - val_categorical_accuracy: 0.2180

Epoch 00004: saving model to simple-examples/data/model-04.hdf5
Epoch 5/20
1549/1549 [==============================] - 1581

## evaluate model

In [31]:
model = load_model(data_path + "/model-{}.hdf5".format(str(num_epochs)))
dummy_iters = 40
example_training_generator = KerasBatchGenerator(train_data, num_steps, 1, vocabulary,
                                                 skip_step=1)
print("Training data:")
for i in range(dummy_iters):
    dummy = next(example_training_generator.generate())
num_predict = 10
true_print_out = "Actual words: "
pred_print_out = "Predicted words: "
for i in range(num_predict):
    data = next(example_training_generator.generate())
    prediction = model.predict(data[0])
    predict_word = np.argmax(prediction[:, num_steps-1, :])
    true_print_out += reversed_dictionary[train_data[num_steps + dummy_iters + i]] + " "
    pred_print_out += reversed_dictionary[predict_word] + " "
print(true_print_out)
print(pred_print_out)
# test data set
dummy_iters = 40
example_test_generator = KerasBatchGenerator(test_data, num_steps, 1, vocabulary,
                                                 skip_step=1)
print("Test data:")
for i in range(dummy_iters):
    dummy = next(example_test_generator.generate())
num_predict = 10
true_print_out = "Actual words: "
pred_print_out = "Predicted words: "
for i in range(num_predict):
    data = next(example_test_generator.generate())
    prediction = model.predict(data[0])
    predict_word = np.argmax(prediction[:, num_steps - 1, :])
    true_print_out += reversed_dictionary[test_data[num_steps + dummy_iters + i]] + " "
    pred_print_out += reversed_dictionary[predict_word] + " "
print(true_print_out)
print(pred_print_out)


Training data:
Actual words: director of this british industrial conglomerate <eos> a form of 
Predicted words: director of this company company company <eos> the <unk> of 
Test data:
Actual words: futures <eos> the N stock specialist firms on the big 
Predicted words: the <eos> the <unk> <unk> market firms are the <unk> 
